# Saving Models with BentoML

In [1]:
# BentoML 가져와
import bentoml
import xgboost as xgb
# Toy Dataset 가져와
from sklearn import datasets

In [2]:
# Breast Cancer 데이터셋 가져오기.
breast_cancer = datasets.load_breast_cancer()
X, y = breast_cancer.data, breast_cancer.target

In [3]:
# 학습된 xgb 모델 만들기.
xgb_clf = xgb.DMatrix(X, label=y)
param = {"max_depth": 3, "eta": 0.3, "objective": "multi:softprob", "num_class": 2}
fitted_clf = xgb.train(param, xgb_clf)

In [28]:
# BentoML(XGBoost) 저장하기
saved_model = bentoml.xgboost.save_model('breast_cancer_clf', fitted_clf)
print(f'Model saved: {saved_model.tag}')

Model saved: breast_cancer_clf:t4rlhfqypgtdjfqt


In [5]:
# 저장되어있는 최신버전 모델 가져오기
bento_model = bentoml.xgboost.load_model("breast_cancer_clf:latest")

# Model to Runner, Runner to Service

In [14]:
import bentoml
from bentoml.io import JSON

In [12]:
# Model을 Runner로
breast_runner = bentoml.xgboost.get('breast_cancer_clf:latest').to_runner()

In [13]:
# Runner를 Service로
bento_service = bentoml.Service("breast_cancer_clf_service", runners=[breast_runner])

In [15]:
# Service에 Custom API 추가
@bento_service.api(input=JSON(), output=JSON())
def predict(input_data: dict) -> dict:
    res = dict()
    res['y_pred'] = breast_runner.predict.run(input_data)
    return res